In [3]:
from IPython.display import display
from ipywidgets import widgets, interactive


%run filterTrailList.ipynb
%run scrapeAllTrails.ipynb
# %run scrapeColorado14ers.ipynb
%run match14ers.ipynb
%run filter14ersbyClass.ipynb
%run getCO14ers.ipynb


layout = widgets.Layout(width='400px', height='20px')

street = widgets.Text(
    value='333 E Colorado Ave',
    placeholder='Type something',
    description='Street Address:',
    disabled=False,
    style= {'description_width': 'initial'},
)
city = widgets.Text(
    value='Colorado Springs',
    placeholder='Type something',
    description='City:',
    disabled=False
)
state = widgets.Text(
    value='CO',
    placeholder='Type something',
    description='State:',
    disabled=False
)
zip_code = widgets.Text(
    value='80903',
    placeholder='Type something',
    description='Zip Code:',
    disabled=False
)
display(street)
display(city)
display(state)
display(zip_code)



slider1 = widgets.FloatRangeSlider(
    min=0,
    max=30,
    step= 0.25,
    display='flex',
    description='Hiking Distance (miles):',
    style= {'description_width': 'initial'},
    layout = layout,
    value = [2, 15]
)
display(slider1)

slider2 = widgets.FloatSlider(
    min=0,
    max=7,
    step= 0.25,
    description='Driving Distance (hours):',
    style= {'description_width': 'initial'},
    layout = layout,
    value = 3.0
)
display(slider2)
slider3 = widgets.FloatSlider(
    min=1,
    max=4,
    step= .5,
    description='Class Limit (for 14ers only):',
    style= {'description_width': 'initial'},
    layout = layout,
    value = 2
)
display(slider3)

feature1 = widgets.RadioButtons(
    options=['Lake','Wildflowers', 'River','Views','Dogs allowed', 'Scramble','Waterfall', 'Forest', 'Wildlife','Trail Running'],
    value='Wildflowers',
    description='Feature #1:',
)

feature2 = widgets.RadioButtons(
    options=['Lake','Wildflowers', 'River','Views','Dogs allowed', 'Scramble','Waterfall', 'Forest', 'Wildlife','Trail Running'],
    value='Lake',
    description='Feature #2:',
)
feature_e = widgets.RadioButtons(
    options=['Snow','No Shade','Scramble','Fee','Forest','None'],
    value='None',
    description='Exclude Trails With:',
    style= {'description_width': 'initial'}
)
display(feature1)
display(feature2)
display(feature_e)

curr_date = date.today()
today = str(calendar.day_name[curr_date.weekday()])
doys = {'Sunday': 0, 'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6}
seq = [0,1,2,3,4,5,6,0,-1,-2,-3,-4,-5,-6]
sel = seq[doys[today]:doys[today]+6]
res =  [abs(ele) for ele in sel]
doy_names = list(doys)
name_opts = [doy_names[i] for i in res]

weekday = widgets.Dropdown(
    options=name_opts,
    description='Hike Day:',
)
display(weekday)

hike_list = widgets.Dropdown(
    options=['All Colorado','Colorado short list','All 14ers','Top 14ers'],
    value='Colorado short list',
    description='Trail List:',
)
display(hike_list)

weather_check = widgets.RadioButtons(
    options=['Yes','No'],
    value='Yes',
    description='Check for Weather:',
    style= {'description_width': 'initial'}
)
display(weather_check)


feature_dict = {'Lake':'lake','Wildflowers':'wild-flowers', 'River':'river','Views':'views',\
                'Dogs allowed':'dogs','Scramble':'scramble','Waterfall':'waterfall', 'Forest':'forest', \
                'Wildlife':'wildlife','Trail Running':'running'}
excluded_dict = {'No Shade':'no-shade','Snow':'snow','None':'nothing','Fee':'fee','Scramble':'scramble', 'Forest':'forest'}
# feature_requests = [feature_dict[feature1.value], feature_dict[feature2.value]]

# address = street.value + ' '+ city.value + ', '+state.value +' '+ zip_code.value

# hike_day = dow.value
# dow = dow.value
# distance_max = slider1.value[1]
# distance_min = slider1.value[0]

# drive_lim = slider2.value
# class_limit = slider3.value

# trail_list = hike_list.value

# check_weather = True

output = widgets.Output()
done = widgets.Button(
    description='Done',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
output = widgets.Output()
display(done, output)

def on_button_clicked(b):
    with output:

        feature_requests = [feature_dict[feature1.value], feature_dict[feature2.value]]
        
        excluded = excluded_dict[feature_e.value]

        address = street.value + ' '+ city.value + ', '+state.value +' '+ zip_code.value

        hike_day = weekday.value
        dow = hike_day
        distance_max = slider1.value[1]
        distance_min = slider1.value[0]

        drive_lim = slider2.value
        class_limit = slider3.value

        trail_list = hike_list.value
        
        if weather_check.value == 'Yes':
            check_weather = True
        else:
            check_weather = False

        full_df = allTrailsAny(address, trail_list, hike_day)
        # full_df.to_csv('check_err.csv')

        full_df_in= full_df[:][:]


        filtered_hikes_df = filterHikes(full_df_in[:][:],distance_max,distance_min,drive_lim,feature_requests,excluded, check_weather,dow)
        # filtered_hikes_df.to_csv('filtered_hikes_'+str(date.today())+'.csv')

        if trail_list == 'Top 14ers' or  trail_list == 'All 14ers':

            all14erData = getallCO14erData()

            co_14_df = all14erData[:][:]
            at_df = filtered_hikes_df[:][:]

            merged_14er_data = get14erMatches(at_df,co_14_df)

            filtered_14er_df = filter14ers(merged_14er_data,class_limit)

            filtered_14er_df.to_csv('filtered_14ers_'+str(date.today())+'.csv')
            print('Analysis Complete.')
        else:
            
            filtered_hikes_df.to_csv('filtered_hikes_'+str(date.today())+'.csv')
            print('Analysis Complete.')

        
done.on_click(on_button_clicked)

Text(value='333 E Colorado Ave', description='Street Address:', placeholder='Type something', style=Descriptio…

Text(value='Colorado Springs', description='City:', placeholder='Type something')

Text(value='CO', description='State:', placeholder='Type something')

Text(value='80903', description='Zip Code:', placeholder='Type something')

FloatRangeSlider(value=(2.0, 15.0), description='Hiking Distance (miles):', layout=Layout(height='20px', width…

...

Dropdown(description='Hike Day:', options=('Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'Monday'),…

Dropdown(description='Trail List:', index=1, options=('All Colorado', 'Colorado short list', 'All 14ers', 'Top…

RadioButtons(description='Check for Weather:', options=('Yes', 'No'), style=DescriptionStyle(description_width…

Button(description='Done', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()

'Forest'

'forest'